# 트리의 앙상블
## 정형 데이터와 비정형 데이터

- 정형 데이터를 다루는 데  가장 뛰어난 성과를 내는 알고리즘이 앙상블 학습(ensemble learning)

- 앙상블 학습은 대부분 결정 트리를 기반으로 만들어져 있음.

- 비정형 데이터에는 신경망 알고리즘을 사용.


## 랜덤 포레스트

- 앙상블 학습의 대표 주자 중 하나로 안정적인 성능을 갖춤.

- 결정 트리를 랜덤하게 만들어 결정 트리(나무)의 숲을 만듭니다. 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만듬.

- 랜덤 포레스트는 각 트리를 훈련하기 위한 데이터를 랜덤하게 만듬. 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만듦. 이때 한 샘플이 중복되어 추출될 수 있음.

- EX) 1000개 구슬이 든 가방에서 100개씩 샘플을 뽑는다면 먼저 1개를 뽑고 뽑았던 1개를 다시 가방에 넣고 다음 걸 뽑음 (복원추출). => "부트스트랩 샘플(Bootstrap sample)"

- 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾는다.

- 분류 모델인 RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택.

- 회귀 모델인 RandomForestRegressor는 전체 특성을 사용.

- 랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합 되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다.

In [1]:
# 사이킷런의 RandomForestClassifier 클래스를 화이트 와인을 분류하는 문제에 적용해보자.
# 와인 데이터셋을 판다스로 불러오고 훈련 세트와 테스트 세트로 나누기
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine  = pd.read_csv('./db/wine_csv_data.csv')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


- RandomForestClassifier에 자체적으로 모델을 평가하는 점수를 얻을 수 있음.

- 랜덤 포레스트는 훈련 세트에서 중복을 허용하여 부트스트랩 샘플을 만들어 결정트리를 훈련. 이때 부트스트랩 샘플에 포함되지 않고 남는 샘플이 있음. 이를 OOB(Out Of Bag) 샘플이라고 함.

- 이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있음. 검증 세트처럼!

In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라 트리

- 